In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 45.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=6f054e75921482a5eb5197ae0974878484046362cc9ce06ad6d498c3fb05e75a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


***Przewidywanie wzięcia pożyczki. ***

Celem projektu jest zastosowanie modeli  klasyfikacji (binarnej) do  przewidzenia czy pożyczka zostanie udzielona danemu klientowi z danymi parametrami wniosku. 

Zmienna celu (wypłacenie pożyczki): ***Disbursed ***


---



***Wymogi:*** 

● Zmienna LoggedIn nie może być uwzględniona w modelowaniu - należy ją od razu wyrzucić ze  zbioru.  
● Nie usuwamy żadnych obserwacji - braki danych uzupełniamy w jakiś sposób.  
● Wszelkie nieoczywiste ​ przekształcenia danych należy opatrzyć uzasadnieniem​ dlaczego  dokonujemy tego przekształcenia (np. robimy wykres i uzasadniamy wzięci logarytmu ze  zmiennej tym, że rozkład jest skośny).  
● Należy wykorzystać przynajmniej 3 metody klasyfikacji.  
● Należy uwzględnić optymalizację modeli/pipelinów.  
● Projekt musi być zakończony przejrzystym porównaniem przetestowanych rozwiązań  (modeli/pipelinów) ​ w postaci tabeli​ , gdzie jeden wiersz opisuje jedno rozwiązanie i jego  wynik. 



Link do danych:  https://raw.githubusercontent.com/saimadhu-polamuri/DataHakthon3X/master/dataSet/Train.csv 


Opis danych (najdokładniejszy jaki istnieje):  https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802 

**IMPORT BIBLIOTEK**

In [2]:
import pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName('my_app').getOrCreate()

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import feature

In [5]:
spark

***Import Danych, Przekształcenia***

In [7]:
csv_df = spark.read.csv('/content/Train.csv', header=True, inferSchema=True)


In [8]:
csv_df.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+--------+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|LoggedIn|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------

Wyrzucenie ze zbioru zmiennej **LoggedIn**

In [9]:
tmp = csv_df.drop('LoggedIn')
tmp.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+

In [10]:
tmp.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = true)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = true)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = true)
 |-- Filled_Form: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Var2: string (nullable = true)
 |-- Source: string (nul

In [11]:
tmp.describe().show()

+-------+-----------+------+--------------+------------------+---------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+---------------+-----------------+-----+---------------------+---------------------+------------------+------------------+------------------+-----------+-----------+-----+------+------------------+--------------------+
|summary|         ID|Gender|          City|    Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|      Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|             Var5| Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|     Interest_Rate|    Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type| Var2|Source|              Var4|           Disbursed|
+-------+-----------+------+--------------+------------------+---------+------------------+-------------------+-------------------+------------------+--------------------+-------

In [12]:
# zmieniamy Null na 0

tmp = tmp.fillna(0)
tmp.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+

In [13]:
tmp.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = false)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = false)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = false)
 |-- Filled_Form: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Var2: string (nullable = true)
 |-- Source: string (

In [14]:
tmp.groupBy("Disbursed").count().show()

+---------+-----+
|Disbursed|count|
+---------+-----+
|        1| 1184|
|        0|78062|
+---------+-----+



In [15]:
#tmp.toPandas()


In [16]:
# Liczba pożyczek
tmp.select((tmp["Disbursed"]).alias("cln")).describe().show()

+-------+--------------------+
|summary|                 cln|
+-------+--------------------+
|  count|               79246|
|   mean|0.014940817202130075|
| stddev| 0.12131683686151994|
|    min|                   0|
|    max|                   1|
+-------+--------------------+



In [17]:
tmp.createOrReplaceTempView("tmp")

In [18]:
# max/min/avg udzielonej pożyczki

spark.sql('select max(Loan_Amount_Applied) as Max_Loan_Amount, min(Loan_Amount_Applied) as Min_Loan_Amount, avg(Loan_Amount_Applied) as Avg_Loan_Amout \
      from tmp \
      where Disbursed == 1').show()

+---------------+---------------+------------------+
|Max_Loan_Amount|Min_Loan_Amount|    Avg_Loan_Amout|
+---------------+---------------+------------------+
|        3000000|              0|321355.58699324325|
+---------------+---------------+------------------+



In [19]:
# sprawdzenie unikalności danych
tmp.groupBy("Device_Type").count().show()
tmp.groupBy("Gender").count().show()
tmp.groupBy("Filled_Form").count().show()
tmp.groupBy("Source").count().show()
tmp.groupBy("Mobile_Verified").count().show()

+-----------+-----+
|Device_Type|count|
+-----------+-----+
|Web-browser|58804|
|     Mobile|20442|
+-----------+-----+

+------+-----+
|Gender|count|
+------+-----+
|Female|33638|
|  Male|45608|
+------+-----+

+-----------+-----+
|Filled_Form|count|
+-----------+-----+
|          Y|17705|
|          N|61541|
+-----------+-----+

+------+-----+
|Source|count|
+------+-----+
|  S155|    4|
|  S143| 4332|
|  S158|  208|
|  S151|  720|
|  S162|   36|
|  S136|    3|
|  S157|  650|
|  S160|   11|
|  S159| 5599|
|  S161|  769|
|  S129|    3|
|  S135|    2|
|  S125|    1|
|  S156|  308|
|  S154|    1|
|  S150|   10|
|  S138|    3|
|  S122|30793|
|  S133|29885|
|  S123|   73|
+------+-----+
only showing top 20 rows

+---------------+-----+
|Mobile_Verified|count|
+---------------+-----+
|              Y|51506|
|              N|27740|
+---------------+-----+



Konwersja danych

In [20]:
from pyspark.sql.functions import col, when

In [75]:
tmp1 = tmp.select(col("*"), when(col("Gender") == "Male",1)
      .when(col("Gender") == "Female",0).alias("new_gender"), when(col("Filled_Form") == "Y",1)
      .when(col("Filled_Form") == "N",0).alias("new_Filled_Form"), when(col("Device_Type") == "Web-browser",1)
      .when(col("Device_Type") == "Mobile",0).alias("new_Device_Type"), when(col("Mobile_Verified") == "Y",1)
      .when(col("Mobile_Verified") == "N",0).alias("new_Mobile_Verified"))
tmp1.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+----------+---------------+---------------+-------------------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+

In [76]:
tmp2 = tmp1.drop("Gender", "Mobile_Veified", "Device_Type", "ID", "Filled_Form")
tmp2.show()


+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+---

In [77]:
tmp2.select("Employer_Name", "Disbursed").groupBy("Employer_Name").sum().orderBy(f.desc("sum(Disbursed)")).show()

+--------------------+--------------+
|       Employer_Name|sum(Disbursed)|
+--------------------+--------------+
|TATA CONSULTANCY ...|            20|
|ACCENTURE SERVICE...|            10|
|COGNIZANT TECHNOL...|             9|
|      INDIAN RAILWAY|             7|
|             GENPACT|             7|
|HCL TECHNOLOGIES LTD|             7|
|  HCL AVITAS PVT LTD|             5|
|           CIPLA LTD|             5|
|ERICSSON INDIA GL...|             4|
|              OTHERS|             4|
|HSBC ELECTRONIC D...|             4|
|   IDEA CELLULAR LTD|             4|
|                   0|             4|
|TCS AND ASSOCIATE...|             4|
|TYPE SLOWLY FOR A...|             4|
|CAPGEMINI INDIA P...|             4|
|     IBM CORPORATION|             4|
|IBM GLOBAL SERVIC...|             4|
|    ABC AND SONS LTD|             3|
|      HCL COMNET LTD|             3|
+--------------------+--------------+
only showing top 20 rows



In [78]:
tmp2 = tmp2.fillna(0)
tmp2.show()

+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+---

In [79]:
df_t, df_e = tmp2.randomSplit([0.7, 0.3], 42)

In [80]:
df_t.show()

+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|    City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+

In [81]:
tmp2.printSchema()

root
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = false)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = false)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = false)
 |-- Var2: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Var4: integer (nullable = true)
 |-- Disbursed: integer (nullable = true)
 |-- new_gender: integer (nullable = true)
 |-- new_Filled_For

In [82]:
tmp2.show(3, vertical=True)

-RECORD 0-------------------------------------
 City                  | Delhi                
 Monthly_Income        | 20000                
 DOB                   | 23-May-78            
 Lead_Creation_Date    | 15-May-15            
 Loan_Amount_Applied   | 300000               
 Loan_Tenure_Applied   | 5                    
 Existing_EMI          | 0.0                  
 Employer_Name         | CYBOSOL              
 Salary_Account        | HDFC Bank            
 Mobile_Verified       | N                    
 Var5                  | 0                    
 Var1                  | HBXX                 
 Loan_Amount_Submitted | 0                    
 Loan_Tenure_Submitted | 0                    
 Interest_Rate         | 0.0                  
 Processing_Fee        | 0                    
 EMI_Loan_Submitted    | 0.0                  
 Var2                  | G                    
 Source                | S122                 
 Var4                  | 1                    
 Disbursed   

In [83]:
formula = ("Disbursed ~ new_gender + new_Device_Type + new_Mobile_Verified + Var1 + new_Filled_Form + Var2 + Source +"
                              "Monthly_Income + Loan_Amount_Applied + Loan_Tenure_Applied")
rf = feature.RFormula(formula=formula, featuresCol='featuresFull')

In [84]:
rfModel = rf.fit(tmp2)

In [85]:
tmp3 = rfModel.transform(tmp2)
tmp3.show(truncate=False)

+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------------------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+-----------------------------------------------------------------------------+-----+
|City       |Monthly_Income|DOB      |Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|Employer_Name                               |Salary_Account      |Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|featuresFull                                                                 |label|
+-----------+--------------+---------+------------------+-----

In [88]:
#Skalowanie
scaler = feature.StandardScaler(inputCol="featuresFull", outputCol="featuresScal")

In [89]:
scalerModel = scaler.fit(tmp3)

In [90]:
tmp3.select("featuresFull").show(truncate=False)
scalerModel.transform(tmp3).select("featuresScal").show(truncate=False)

+-----------------------------------------------------------------------------+
|featuresFull                                                                 |
+-----------------------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |
|(60,[0,1,2,9,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|
|(60,[0,1,2,3,22,34,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,100000.0,500000.0,2.0])|
|(60,[0,1,2,15,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,45000.0,300000.0,5.0])|
|(60,[1,3,22,29,57,58,59],[1.0,1.0,1.0,1.0,70000.0,6.0,5.0])                  |
|(60,[0,1,2,3,22,30,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,20000.0,200000.0,5.0]) |
|(60,[0,2,7,21,24,28,57],[1.0,1.0,1.0,1.0,1.0,1.0,75000.0])                   |
|(60,[1,2,4,22,29,57,58,59],[1.0,1.0,1.0

In [91]:
MMscaler = feature.MinMaxScaler(inputCol="featuresFull", outputCol="featuresScal")

In [92]:
MMscalerModel = MMscaler.fit(tmp3)

In [93]:
tmp3.select("featuresFull").show(truncate=False)
MMscalerModel.transform(tmp3).select("featuresScal").show(truncate=False)

+-----------------------------------------------------------------------------+
|featuresFull                                                                 |
+-----------------------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |
|(60,[0,1,2,9,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|
|(60,[0,1,2,3,22,34,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,100000.0,500000.0,2.0])|
|(60,[0,1,2,15,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,45000.0,300000.0,5.0])|
|(60,[1,3,22,29,57,58,59],[1.0,1.0,1.0,1.0,70000.0,6.0,5.0])                  |
|(60,[0,1,2,3,22,30,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,20000.0,200000.0,5.0]) |
|(60,[0,2,7,21,24,28,57],[1.0,1.0,1.0,1.0,1.0,1.0,75000.0])                   |
|(60,[1,2,4,22,29,57,58,59],[1.0,1.0,1.0

**PCA**

In [94]:
pca = feature.PCA(k=3, inputCol="featuresFull", outputCol="featuresPCA")

In [95]:
PCAmodel = pca.fit(tmp3)

In [96]:
PCAmodel.transform(tmp3).select("featuresFull", "featuresPCA").show(truncate=False)

+-----------------------------------------------------------------------------+------------------------------------------------------------+
|featuresFull                                                                 |featuresPCA                                                 |
+-----------------------------------------------------------------------------+------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |[-20158.30434071145,-299989.4044364668,4.080184413672564]   |
|(60,[0,1,2,9,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |[-35105.533210530804,-199981.50299475883,1.3703828477141546]|
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |[-22816.611117949764,-599988.0434381654,2.3364751336023284] |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|[-35527.68554465821,-999981.3916176226,2.1356377551430494]  |
|(60,[0,1,2,3

In [97]:
PCAmodel.explainedVariance

DenseVector([0.976, 0.024, 0.0])

In [98]:
rfModel = rf.fit(tmp2)

In [99]:
df_train = rfModel.transform(df_t)
df_eval = rfModel.transform(df_e)

In [100]:
scaler = feature.StandardScaler(inputCol="featuresFull", outputCol="features")
scal_mod = scaler.fit(df_train)
df_train = scal_mod.transform(df_train)
df_eval = scal_mod.transform(df_eval)

df_eval.show()

+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+--------------------+-----+--------------------+
|    City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|        featuresFull|label|            features|
+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+-------------

In [101]:
num_cols = [c for c, t in df_t.dtypes if t != "string"]
categ_cols = [c for c, t in df_t.dtypes if t == "string" and c != "Disbursed"]
categ_cols_idx = [c + "Idx" for c in categ_cols]
categ_cols_vect = [c + "Vect" for c in categ_cols]

In [102]:
indexer = feature.StringIndexer(inputCol="Disbursed", outputCol="label1")
indexers = feature.StringIndexer(inputCols=categ_cols, outputCols=categ_cols_idx)
OHencoder = feature.OneHotEncoder(inputCols=categ_cols_idx, outputCols=categ_cols_vect)
vectAssembler = feature.VectorAssembler(inputCols = num_cols + categ_cols_vect, outputCol = "featuresRaw")
#scaler = feature.StandardScaler(inputCol="featuresRaw", outputCol="features")

In [103]:
pipe = Pipeline(stages=[indexer, indexers, OHencoder, vectAssembler, scaler])
pipeModel = pipe.fit(tmp3)

In [104]:
df_train = pipeModel.transform(tmp3)
df_eval = pipeModel.transform(tmp3)

In [ ]:
df_eval.show()

In [105]:
df_train = df_train.select("label", "features")
df_eval = df_eval.select("label", "features")

In [ ]:
print("Train:")
df_train.groupBy("label").count().show()
print("Eval:")
df_eval.groupBy("label").count().show()

In [107]:
df_train.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(60,[1,3,23,28,57...|
|  0.0|(60,[0,1,2,9,23,2...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,15,22,...|
|  0.0|(60,[1,3,22,29,57...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,2,7,21,24,...|
|  0.0|(60,[1,2,4,22,29,...|
|  0.0|(60,[0,1,2,4,22,2...|
|  0.0|(60,[1,3,22,28,57...|
|  0.0|(60,[1,3,22,29,57...|
|  0.0|(60,[0,1,2,8,22,2...|
|  0.0|(60,[1,3,22,36,57...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,11,25,...|
|  0.0|(60,[1,3,22,28,57...|
|  0.0|(60,[1,3,25,29,57...|
|  0.0|(60,[0,1,2,3,25,2...|
+-----+--------------------+
only showing top 20 rows



In [108]:
#REGRESJA LINIOWA

In [109]:
from pyspark.ml import classification

In [110]:
lr = classification.LogisticRegression(maxIter=1000)

In [111]:
lrModel = lr.fit(df_train)

In [112]:
lrModel.coefficients

DenseVector([0.1697, -0.0563, 0.2341, 0.8137, 0.2944, 0.0188, 0.3369, 0.4229, 0.307, 0.3599, 0.3713, 0.104, 0.1843, 0.0955, 0.1112, 0.1373, 0.0466, 0.0455, -3.6746, 0.061, -2.3379, 0.1917, -0.2612, -0.4866, -0.2366, -0.1025, -7.301, -0.0919, -0.2066, -0.4005, -0.3647, -0.1552, -0.247, -0.7104, -0.015, -0.2652, -0.1885, -0.1576, -0.2044, -4.9551, -0.1065, -0.1032, -2.3739, -0.0228, -1.664, -1.3544, 0.0158, -0.8366, -0.5732, -0.4834, -0.4913, -0.4597, -0.5082, -0.278, -0.2401, -0.2733, -0.2847, -0.0173, 0.0878, 0.3159])

In [113]:
lrModel.intercept

-5.541862780721203

In [114]:
trainingSummary = lrModel.summary
trainingSummary

In [115]:
trainingSummary.roc.show(120)

+--------------------+--------------------+
|                 FPR|                 TPR|
+--------------------+--------------------+
|                 0.0|                 0.0|
|0.005162563090876483|0.030405405405405407|
| 0.01044041915400579| 0.07179054054054054|
| 0.01648689503215393| 0.10641891891891891|
|0.022020957700289515| 0.12837837837837837|
|0.028003381927186082|  0.1554054054054054|
|0.035330890830365606| 0.18496621621621623|
| 0.04228690015628603| 0.20523648648648649|
| 0.04801311777817632| 0.22888513513513514|
| 0.05441828290333325|  0.2592905405405405|
| 0.06433347851707617| 0.29307432432432434|
| 0.07123824652199534| 0.30067567567567566|
| 0.07727191206989316| 0.31841216216216217|
| 0.08538085111834183| 0.34121621621621623|
| 0.09325920422228486| 0.35557432432432434|
| 0.09947221439368707|  0.3766891891891892|
| 0.11260280290025877|  0.3910472972972973|
| 0.11927698496067228| 0.40202702702702703|
| 0.12803925085188697| 0.41385135135135137|
| 0.13690399938510414| 0.4315878

In [116]:
trainingSummary.roc.toPandas()

,FPR,TPR
0,0.000000,0.000000
1,0.005163,0.030405
2,0.010440,0.071791
3,0.016487,0.106419
4,0.022021,0.128378
...,...,...
98,0.977403,0.999155
99,0.984999,1.000000
100,0.991904,1.000000
101,1.000000,1.000000


In [117]:
trainingSummary.pr.show(120)

+--------------------+--------------------+
|              recall|           precision|
+--------------------+--------------------+
|                 0.0| 0.08200455580865604|
|0.030405405405405407| 0.08200455580865604|
| 0.07179054054054054| 0.09444444444444444|
| 0.10641891891891891| 0.08917197452229299|
| 0.12837837837837837| 0.08123997862105825|
|  0.1554054054054054| 0.07763713080168777|
| 0.18496621621621623| 0.07356399059455827|
| 0.20523648648648649|  0.0685665914221219|
| 0.22888513513513514| 0.06742970888280667|
|  0.2592905405405405| 0.06739846322722283|
| 0.29307432432432434| 0.06463028496926802|
| 0.30067567567567566| 0.06016562447186074|
| 0.31841216216216217|0.058823529411764705|
| 0.34121621621621623|0.057150940727118404|
| 0.35557432432432434|0.054668224905856384|
|  0.3766891891891892| 0.05431737912556327|
|  0.3910472972972973|0.050037825570085376|
| 0.40202702702702703|  0.0486359456421784|
| 0.41385135135135137| 0.04673342870767763|
| 0.43158783783783783|0.04563314

In [118]:
trainingSummary.areaUnderROC

0.7426757531868292

In [119]:
trainingSummary.accuracy

0.9850465638644221

In [120]:
trainingSummary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(60,[1,3,23,28,57...|[4.52893029620545...|[0.98932301394928...|       0.0|
|  0.0|(60,[0,1,2,9,23,2...|[3.44250890370919...|[0.96900695316251...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[3.51551344611666...|[0.97112596521096...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[3.25936071318807...|[0.96300802376454...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[3.29031707276910...|[0.96409513143118...|       0.0|
|  0.0|(60,[0,1,2,15,22,...|[2.71518286695016...|[0.93791662776904...|       0.0|
|  0.0|(60,[1,3,22,29,57...|[4.48528041771132...|[0.98885195428468...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[4.19825877845019...|[0.98520060207913...|       0.0|
|  0.0|(60,[0,2,7,21,24,...|[2.51572044705789...|[0.92523655980222...|       0.0|
|  0.0|(60,[1,2,

In [121]:
svm = classification.LinearSVC(maxIter=1000)

In [122]:
svmModel = svm.fit(df_train)

In [123]:
svmModel.coefficients

DenseVector([0.0135, -0.0586, -0.126, -0.1923, -0.0549, -0.0369, -0.0323, -0.027, -0.0268, -0.0255, -0.0229, -0.0184, -0.0163, -0.0134, -0.012, -0.0098, -0.0091, -0.0086, -0.0081, -0.0062, -0.0052, -0.0001, -0.0475, -0.0995, -0.0874, -0.0122, -0.0174, -0.0188, -0.0943, -0.0869, -0.0459, -0.0407, -0.0276, -0.0262, -0.0228, -0.0178, -0.017, -0.0162, -0.0141, -0.0132, -0.011, -0.0092, -0.0064, -0.0048, -0.0041, -0.0033, -0.0021, -0.0021, -0.0015, -0.0028, -0.0012, -0.0033, -0.0011, -0.001, -0.0015, -0.0008, -0.0008, 0.0, -0.0, 0.0005])

In [124]:
svmModel.intercept

-0.17921073436598828

In [125]:
svmModel.transform(df_eval).show()

+-----+--------------------+--------------------+----------+
|label|            features|       rawPrediction|prediction|
+-----+--------------------+--------------------+----------+
|  0.0|(60,[1,3,23,28,57...|[1.13266117198462...|       0.0|
|  0.0|(60,[0,1,2,9,23,2...|[1.13323515354294...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[1.23710325623638...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[1.23686512553237...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[1.23758211818126...|       0.0|
|  0.0|(60,[0,1,2,15,22,...|[1.00000466269413...|       0.0|
|  0.0|(60,[1,3,22,29,57...|[1.00000571439930...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[1.23688388784783...|       0.0|
|  0.0|(60,[0,2,7,21,24,...|[1.01421686134092...|       0.0|
|  0.0|(60,[1,2,4,22,29,...|[1.02783488622622...|       0.0|
|  0.0|(60,[0,1,2,4,22,2...|[1.00001111983164...|       0.0|
|  0.0|(60,[1,3,22,28,57...|[1.01445984616857...|       0.0|
|  0.0|(60,[1,3,22,29,57...|[1.00024592495157...|       0.0|
|  0.0|(60,[0,1,2,8,22,2

DRZEWO DECYZYJNE

In [126]:
tree = classification.DecisionTreeClassifier()

In [127]:
treeModel = tree.fit(df_train)

In [128]:
treeModel.depth

5

In [129]:
treeModel.numNodes

15

In [130]:
print(treeModel.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_8c87164e2d4c, depth=5, numNodes=15, numClasses=2, numFeatures=60
  If (feature 57 <= 0.015542318662684716)
   Predict: 0.0
  Else (feature 57 > 0.015542318662684716)
   If (feature 33 <= 3.4273897943968925)
    If (feature 21 <= 1.2003679031581753)
     Predict: 0.0
    Else (feature 21 > 1.2003679031581753)
     If (feature 59 <= 0.7452165304263637)
      Predict: 0.0
     Else (feature 59 > 0.7452165304263637)
      If (feature 1 <= 1.1428243567584642)
       Predict: 1.0
      Else (feature 1 > 1.1428243567584642)
       Predict: 0.0
   Else (feature 33 > 3.4273897943968925)
    If (feature 57 <= 0.04848653095083681)
     Predict: 0.0
    Else (feature 57 > 0.04848653095083681)
     If (feature 59 <= 2.235649591279091)
      Predict: 0.0
     Else (feature 59 > 2.235649591279091)
      Predict: 1.0



RANDOM FORREST

In [131]:
forest = classification.RandomForestClassifier()

In [132]:
forestModel = forest.fit(df_train)

In [133]:
forestModel.featureImportances

SparseVector(60, {0: 0.0091, 1: 0.0175, 2: 0.0317, 3: 0.0191, 4: 0.017, 5: 0.0242, 6: 0.004, 7: 0.0368, 8: 0.0217, 9: 0.0196, 10: 0.0911, 15: 0.0102, 21: 0.0729, 22: 0.0028, 24: 0.0407, 25: 0.0667, 26: 0.0011, 27: 0.0179, 30: 0.004, 34: 0.0133, 43: 0.0034, 46: 0.0347, 57: 0.1807, 58: 0.1422, 59: 0.1177})

In [134]:
print(forestModel.toDebugString)

RandomForestClassificationModel: uid=RandomForestClassifier_2ef3c5cdcd12, numTrees=20, numClasses=2, numFeatures=60
  Tree 0 (weight 1.0):
    If (feature 59 <= 1.7388385709948486)
     Predict: 0.0
    Else (feature 59 > 1.7388385709948486)
     If (feature 7 <= 3.2786193881591643)
      If (feature 4 <= 1.6120399576642956)
       If (feature 10 <= 3.85683737342877)
        Predict: 0.0
       Else (feature 10 > 3.85683737342877)
        If (feature 58 <= 5.655318936141717E-6)
         Predict: 1.0
        Else (feature 58 > 5.655318936141717E-6)
         Predict: 0.0
      Else (feature 4 > 1.6120399576642956)
       Predict: 0.0
     Else (feature 7 > 3.2786193881591643)
      Predict: 0.0
  Tree 1 (weight 1.0):
    Predict: 0.0
  Tree 2 (weight 1.0):
    If (feature 34 <= 3.934701902436823)
     Predict: 0.0
    Else (feature 34 > 3.934701902436823)
     If (feature 57 <= 0.015963053045672297)
      If (feature 7 <= 3.2786193881591643)
       Predict: 0.0
      Else (feature 7 > 3.

Naiwny Bayes

In [137]:
bayes = classification.NaiveBayes()

In [138]:
bayesModel = bayes.fit(df_train)

In [ ]:
PREDYKCJE

In [135]:

treeModel.transform(df_eval).show()


+-----+--------------------+---------------+--------------------+----------+
|label|            features|  rawPrediction|         probability|prediction|
+-----+--------------------+---------------+--------------------+----------+
|  0.0|(60,[1,3,23,28,57...|[56476.0,322.0]|[0.99433078629529...|       0.0|
|  0.0|(60,[0,1,2,9,23,2...|[56476.0,322.0]|[0.99433078629529...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[56476.0,322.0]|[0.99433078629529...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[56476.0,322.0]|[0.99433078629529...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[15881.0,574.0]|[0.96511698571862...|       0.0|
|  0.0|(60,[0,1,2,15,22,...|[15881.0,574.0]|[0.96511698571862...|       0.0|
|  0.0|(60,[1,3,22,29,57...|[15881.0,574.0]|[0.96511698571862...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[56476.0,322.0]|[0.99433078629529...|       0.0|
|  0.0|(60,[0,2,7,21,24,...| [2315.0,112.0]|[0.95385249278945...|       0.0|
|  0.0|(60,[1,2,4,22,29,...|[56476.0,322.0]|[0.99433078629529...|       0.0|

In [136]:
forestModel.transform(df_eval).show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(60,[1,3,23,28,57...|[19.7009913821294...|[0.98504956910647...|       0.0|
|  0.0|(60,[0,1,2,9,23,2...|[19.6356305875425...|[0.98178152937712...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[19.6806361109751...|[0.98403180554875...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[19.6489318135976...|[0.98244659067988...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[19.5569498156255...|[0.97784749078127...|       0.0|
|  0.0|(60,[0,1,2,15,22,...|[19.6291488309325...|[0.98145744154662...|       0.0|
|  0.0|(60,[1,3,22,29,57...|[19.6966208583458...|[0.98483104291729...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[19.6955195246363...|[0.98477597623181...|       0.0|
|  0.0|(60,[0,2,7,21,24,...|[19.6238707295942...|[0.98119353647971...|       0.0|
|  0.0|(60,[1,2,

In [139]:
bayesModel.transform(df_eval).show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(60,[1,3,23,28,57...|[-32.367599319720...|[0.99465476100842...|       0.0|
|  0.0|(60,[0,1,2,9,23,2...|[-65.376974671808...|[0.03767536752379...|       1.0|
|  0.0|(60,[0,1,2,3,22,3...|[-55.279786923082...|[0.95319274609915...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[-60.252236854786...|[0.93751479254097...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[-71.417918611679...|[0.12952716212952...|       1.0|
|  0.0|(60,[0,1,2,15,22,...|[-150.55748060802...|[9.24402237742013...|       1.0|
|  0.0|(60,[1,3,22,29,57...|[-28.813951492051...|[0.99288125040321...|       0.0|
|  0.0|(60,[0,1,2,3,22,3...|[-50.350442208693...|[0.99933938545627...|       0.0|
|  0.0|(60,[0,2,7,21,24,...|[-64.340401419747...|[0.01176899179267...|       1.0|
|  0.0|(60,[1,2,

**ACCURANCY**

In [140]:
def calculate_acc(df, label="label", prediction="prediction"):
    temp = df.select(f.when(df[label] == df[prediction], 1).otherwise(0).alias("same"))
    return temp.select(f.avg("same")).collect()[0][0]

In [145]:
ac_lr = calculate_acc(lrModel.transform(df_eval))
ac_lr

0.9850465638644221

In [147]:
ac_bayes = calculate_acc(bayesModel.transform(df_eval))
ac_bayes

0.9058248996794791

In [148]:
ac_forest = calculate_acc(forestModel.transform(df_eval))
ac_forest

0.98505918279787

In [149]:
ac_tree = calculate_acc(treeModel.transform(df_eval))
ac_tree

0.9850844206647654

In [154]:
wynik = spark.createDataFrame([Row(metoda='Regresja_Liniowa', accuracy=ac_lr),
                                  Row(metoda='Naiwny_Bayes', accuracy=ac_bayes),
                                  Row(metoda='Las_Losowy', accuracy=ac_forest),
                                  Row(metoda='Drzewo_decyzyjne', accuracy=ac_tree)])
wynik

DataFrame[metoda: string, accuracy: double]

In [155]:
wynik.show()

+----------------+------------------+
|          metoda|          accuracy|
+----------------+------------------+
|Regresja_Liniowa|0.9850465638644221|
|    Naiwny_Bayes|0.9058248996794791|
|      Las_Losowy|  0.98505918279787|
|Drzewo_decyzyjne|0.9850844206647654|
+----------------+------------------+



In [156]:
wynik.toPandas()

,metoda,accuracy
0,Regresja_Liniowa,0.985047
1,Naiwny_Bayes,0.905825
2,Las_Losowy,0.985059
3,Drzewo_decyzyjne,0.985084


**Wniosek:**
najlepszym modelem jest las losowy. W tej metodzi mamy najwyższe accuracy oraz najtrafniej przewiduje grupę, której udzielono pożyczkę.Można jeszcze poprawić parametry przy użyciu funkcji *tuning*